A summary of the EEG data I will collect is as follows:

   - Awake, resting EEG (5 mins with eyes open, 5 mins with eyes shut, participant sat in chair)
   - Sleep EEG
   - Behavioural task EEG. The structure of the task is as follows:
        10s ‘rest’ period, participant looks at central cross on screen- halfway through this rest period (e.g. at 5s) a 1s tone will play so that we can get an auditory ERP
        3s ‘countdown’ to task, where participant is instructed that they will either imagine moving their finger or perform a finger movement
    -    15s of task: either imagination or movement of finger
     -   2s ‘trial finished’ screen

Participants will complete 10 trials of imagined finger movements, 10 trials of performing finger movements. Order of trials will be random. In this task I may also enlist a sensor to detect the finger movements, but I am unsure what sensor to use so am still exploring my options.

In [1]:
import mne
import os # python library to deal with filepaths etc..

# Importing data 

In [10]:
filename = '/home/claire/Documents/scripts-local/EEG_tutorial_amber/sample_data/s558_02.vhdr'


# load your raw data, check for the right function if they are not edf files
raw = mne.io.read_raw_brainvision(filename, preload=True)

# you may use one of the following depending on your needs:

# to rename channels:
#raw.rename_channels(mapping={'E1H1\t//EXG1 HE ': 'EOG L', 'E2H2\t//EXG2 HE ': 'EOG R', 'E3LE\t//EXG3 LE ': 'EOG V L', 'E5M2\t//EXG5 M2 ': 'M2', 'E4M1\t//EXG4 M1 ': 'M1' })

# to specifuy channel types, if you have other types than EEG:

#raw.set_channel_types(mapping={'EOG L': 'eog', 'EOG R': 'eog', 'EOG V L': 'eog', 'M1':'misc', 'M2': 'misc'})


Extracting parameters from /home/claire/Documents/scripts-local/EEG_tutorial_amber/sample_data/s558_02.vhdr...
Setting channel info structure...
Reading 0 ... 1707139  =      0.000 ...  1707.139 secs...


## reading events

In [12]:
# with brainvision events (triggers) are saved in a specific marker file (.vmrk)
events, _ = mne.events_from_annotations(raw)

# if your recording system does not save events in a marker file, then they should be in a specific channel and this function should find it:
# events = mne.find_events(raw,stim_channel= auto verbose=True) # could be stim_channel = -1

# you can  save the events in a file. Convention in mne is that event file names to finish with -eve or saved as .eve.# 
mne.write_events(os.path.join('/home/claire/Documents/scripts-local/EEG_tutorial_amber/sample_data/', 's558_s02.eve'), events)



Used Annotations descriptions: ['New Segment/', 'Stimulus/S  8', 'Stimulus/S 11', 'Stimulus/S 13', 'Stimulus/S 21', 'Stimulus/S 22', 'Stimulus/S 25', 'Stimulus/S 44', 'Stimulus/S 50', 'Stimulus/S 55']


## get electrode location

In [ ]:
montage= mne.channels.read_montage('standard_1020', path = '~/mne-python/mne/channels/data/montages/')# change to where you've put you mne python folder
raw.set_montage(montage)

## set reference

In [ ]:
raw.set_eeg_reference(ref_channels='average') # adjust to the kind of reference you want


## filter your data

In [ ]:
# here it is a fir filter, check on mne website for all options
raw_fir = raw.filter(1.,40, fir_design='firwin')


# preprocess continuuous data (for your resting EEG)
To remove electrodes that are off or shows a lot of artefacts just click on the electrode trace and it will be greyed out. To remove portion of continuuous data that presents big artefacts you want to plot the data, then press 'a' to switch to annotation mode. You can then create a new 'bad_' annotation (specify the name you want) and you can select the portion of data to be marked as bad if you left click and drag. The help button provides also all the display option you have

In [ ]:
raw.plot()


access your annotation and check bad data

In [ ]:
# channels labbeled for rejection 
raw.info['bads']

# summary of annotations 
raw.annotations

# access to the details for a specific annotation, gives you onset duration and description of the data
raw.annotations[1]

## use ICA 
check tutorial here: 
https://mne-tools.github.io/dev/auto_tutorials/preprocessing/plot_40_artifact_correction_ica.html?highlight=ica

# preprocess event-related data (ie. for your behavioural task)

 Epoch the data

In [ ]:
# you can define a dictionnary with more informative events id

# only get go and nogo trials
event_id = {'GO':11, 'NOGO':13}

# define the time window before and after the events that you want
tmin = -0.2
tmax = 0.5

#epoch the data for the go and nogo trials only
epochs = mne.Epochs(raw, events, event_id, tmin, tmax,
                    baseline=(None, 0), reject=None,
                    verbose=False, detrend=0,  preload=True)



In [ ]:
# to get a summary of what's in your data:
epochs

In [ ]:
# you can check the epochs, look up the help button for display options
# you can directly mark bad epochs by clicking on them
epochs.plot()

optional auto reject function

In [ ]:
from autoreject import get_rejection_threshold # need to get autoreject https://autoreject.github.io/
 
reject = get_rejection_threshold(epochs)
reject

rej= {'eeg': reject['eeg']}


In [ ]:
you can also use ICA on epoched data

 actually remove bad data:

In [ ]:
epochs.drop_bad()

In [ ]:
you can plot a log of what has been rejected

In [ ]:
epochs.plot_drop_log()

## interpolate bad channels
you can interpolate bad channels on any eeg objecty (ie. raw, epoch, evoked etc.)

In [ ]:
epochs.interpolate_bads() # remov channels marked previously as bad and interpolate them

## resampling

In [ ]:
epochs.decimate(decim =4) # decim from 1000 to 250 to save ram

# Evoked data 

In [ ]:
compute average the epochs for each type of stimuli of interest:

In [ ]:
# 
ep_go = epochs['GO'].average() 
ep_nogo = epochs['NOGO'].average()

plot the ERP for the 2 conditions

In [ ]:
mne.viz.plot_compare_evokeds([ep_go, ep_nogo], picks= 'eeg')

# check ?mne.viz.plot_compare_evokeds for all the options

plot the ERP difference between the 2 conditions

In [ ]:
diff_evoked = mne.combine_evoked([ep_go, -ep_nogo ], weights= 'equal')
mne.viz.plot_compare_evokeds(diff_evoked, picks= 'eeg')